In [1]:
import pandas as pd

In [2]:
from flask import Flask

In [3]:
from sqlalchemy import create_engine, inspect, MetaData, Table
import subprocess



In [4]:
DATABASE_URL = subprocess.check_output("heroku config:get DATABASE_URL --app billboardchart2", shell=True).decode('utf-8')
engine = create_engine(DATABASE_URL)


In [ ]:
metadata = MetaData()
metadata.create_all(engine)


In [ ]:
query_str1 = """SELECT "Song", count(*) AS cnt  
                        FROM billboard_chart 
                        GROUP BY "Song" 
                        ORDER BY cnt DESC"""

query_str2 = """SELECT "Song", count(*) AS cnt  
                        FROM billboard_chart 
                        WHERE ("Week Position" = 1)
                        GROUP BY "Song" 
                        ORDER BY cnt DESC"""


In [18]:
def test():
    x_values = []
    y_values = []
    values_list = []
    with engine.connect() as con:
        # change this to whatever query you want you want
        query_str = """SELECT "Song", count(*) AS cnt  
                        FROM billboard_chart 
                        GROUP BY "Song" 
                        ORDER BY cnt DESC
                        LIMIT 20"""

        rs = con.execute(query_str)

        for row in rs:
            values_list.append({"x_vals":row[1], "y_vals": row[1]})
            

            
    return values_list
            
dictss = test()

In [19]:
import json

type(json.loads(json.dumps(dictss)))

list

In [35]:
from flask import Flask, render_template, jsonify, request, redirect

dict2 = dictss

def create_app():
    app = Flask(__name__)

    with app.app_context():
        res = test()
        #res = json.loads(json.dumps(res))
        return jsonify(data=res)
        #print(res)

    return jsonify(data=res)




response = create_app()
response


<Response 571 bytes [200 OK]>

In [89]:
query2 = """SELECT * FROM billboard_chart LIMIT 20"""
rs = engine.connect().execute(query2)
for row in rs:
    print (row)
    

('http://www.billboard.com/charts/hot-100/1958-08-02', datetime.date(1958, 8, 2), 1, 'Poor Little Fool', 'Ricky Nelson', 'Poor Little FoolRicky Nelson', 1, None, 1, 1)
('http://www.billboard.com/charts/hot-100/1995-12-02', datetime.date(1995, 12, 2), 1, 'One Sweet Day', 'Mariah Carey & Boyz II Men', 'One Sweet DayMariah Carey & Boyz II Men', 1, None, 1, 1)
('http://www.billboard.com/charts/hot-100/1997-10-11', datetime.date(1997, 10, 11), 1, 'Candle In The Wind 1997/Something About The Way You Look Tonight', 'Elton John', 'Candle In The Wind 1997/Something About The Way You Look TonightElton John', 1, None, 1, 1)
('http://www.billboard.com/charts/hot-100/2006-07-01', datetime.date(2006, 7, 1), 1, 'Do I Make You Proud', 'Taylor Hicks', 'Do I Make You ProudTaylor Hicks', 1, None, 1, 1)
('http://www.billboard.com/charts/hot-100/2009-10-24', datetime.date(2009, 10, 24), 1, '3', 'Britney Spears', '3Britney Spears', 1, None, 1, 1)
('http://www.billboard.com/charts/hot-100/2011-02-26', dateti

In [54]:
billboard_chart_table  = Table('billboard_chart', metadata, autoload=True, autoload_with=engine)
columns = [c.name for c in billboard_chart_table.columns]
print(columns)

['url', 'WeekID', 'Week Position', 'Song', 'Performer', 'SongID', 'Instance', 'Previous Week Position', 'Peak Position', 'Weeks on Chart']


In [36]:
billboard_df = pd.read_sql_table("billboard_chart", con=engine)
billboard_df.shape

(320495, 10)

In [37]:
billboard_df.head(100)

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1958-0...,1958-08-02,1,Poor Little Fool,Ricky Nelson,Poor Little FoolRicky Nelson,1,NaN,1,1
1,http://www.billboard.com/charts/hot-100/1995-1...,1995-12-02,1,One Sweet Day,Mariah Carey & Boyz II Men,One Sweet DayMariah Carey & Boyz II Men,1,NaN,1,1
2,http://www.billboard.com/charts/hot-100/1997-1...,1997-10-11,1,Candle In The Wind 1997/Something About The Wa...,Elton John,Candle In The Wind 1997/Something About The Wa...,1,NaN,1,1
3,http://www.billboard.com/charts/hot-100/2006-0...,2006-07-01,1,Do I Make You Proud,Taylor Hicks,Do I Make You ProudTaylor Hicks,1,NaN,1,1
4,http://www.billboard.com/charts/hot-100/2009-1...,2009-10-24,1,3,Britney Spears,3Britney Spears,1,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...
95,http://www.billboard.com/charts/hot-100/1996-0...,1996-07-20,9,Change The World,Eric Clapton,Change The WorldEric Clapton,1,NaN,9,1
96,http://www.billboard.com/charts/hot-100/2008-0...,2008-09-06,9,So What,P!nk,So WhatP!nk,1,NaN,9,1
97,http://www.billboard.com/charts/hot-100/2009-1...,2009-11-14,9,Bad Romance,Lady Gaga,Bad RomanceLady Gaga,1,NaN,9,1
98,http://www.billboard.com/charts/hot-100/2011-0...,2011-01-01,9,6 Foot 7 Foot,Lil Wayne Featuring Cory Gunz,6 Foot 7 FootLil Wayne Featuring Cory Gunz,1,NaN,9,1


In [65]:
new_df = billboard_df[["Song", "SongID"]].groupby("Song")\
                             .count()\
                             .reset_index() \
                             .sort_values(['SongID'], ascending=False) \
                             .rename(columns={"SongID":"count"})\
                             .to_json(orient='records')



new_df

'[{"Song":"Stay","count":207},{"Song":"Angel","count":205},{"Song":"Heaven","count":194},{"Song":"Hold On","count":189},{"Song":"I Like It","count":188},{"Song":"You","count":184},{"Song":"Crazy","count":177},{"Song":"Forever","count":161},{"Song":"Happy","count":157},{"Song":"Please Don\'t Go","count":152},{"Song":"Home","count":152},{"Song":"Someday","count":149},{"Song":"Runaway","count":140},{"Song":"I Love You","count":139},{"Song":"I Want You","count":138},{"Song":"Without You","count":134},{"Song":"Smile","count":132},{"Song":"Call Me","count":130},{"Song":"Love Song","count":128},{"Song":"Perfect","count":128},{"Song":"Always","count":127},{"Song":"Lady","count":127},{"Song":"Let It Go","count":126},{"Song":"Magic","count":124},{"Song":"Closer","count":122},{"Song":"Missing You","count":122},{"Song":"Dangerous","count":122},{"Song":"Breathe","count":122},{"Song":"I Miss You","count":120},{"Song":"Talk To Me","count":118},{"Song":"Because Of You","count":117},{"Song":"Dance With

In [102]:
def get_data(df, rows):

    billboard_sample = df.iloc[:rows]
    result = billboard_sample.to_json(orient='records')
    return result

res = get_data(billboard_df, 5)

In [55]:
def no1_on_chart():
    values_list = []
    con = engine.connect()
    # change this to whatever query you want you want
    query_str = """SELECT "Song", count(*) AS cnt  
                    FROM billboard_chart 
                    GROUP BY "Song" 
                    ORDER BY cnt DESC
                    LIMIT 20"""

    rs = con.execute(query_str)

    for row in rs:
        values_list.append({"x_vals":row[0], "y_vals": row[1]})


    #print (values_list)
    return values_list

In [57]:
s = no1_on_chart()
s

[{'x_vals': 'Stay', 'y_vals': 207},
 {'x_vals': 'Angel', 'y_vals': 205},
 {'x_vals': 'Heaven', 'y_vals': 194},
 {'x_vals': 'Hold On', 'y_vals': 189},
 {'x_vals': 'I Like It', 'y_vals': 188},
 {'x_vals': 'You', 'y_vals': 184},
 {'x_vals': 'Crazy', 'y_vals': 177},
 {'x_vals': 'Forever', 'y_vals': 161},
 {'x_vals': 'Happy', 'y_vals': 157},
 {'x_vals': 'Home', 'y_vals': 152},
 {'x_vals': "Please Don't Go", 'y_vals': 152},
 {'x_vals': 'Someday', 'y_vals': 149},
 {'x_vals': 'Runaway', 'y_vals': 140},
 {'x_vals': 'I Love You', 'y_vals': 139},
 {'x_vals': 'I Want You', 'y_vals': 138},
 {'x_vals': 'Without You', 'y_vals': 134},
 {'x_vals': 'Smile', 'y_vals': 132},
 {'x_vals': 'Call Me', 'y_vals': 130},
 {'x_vals': 'Perfect', 'y_vals': 128},
 {'x_vals': 'Love Song', 'y_vals': 128}]

In [38]:
!pip install pandasql


  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26822 sha256=434fe937cdd7a55e69bf21b7278488d71b35a6c85aeab217cbde2707e8efede5
  Stored in directory: c:\users\glory\appdata\local\pip\cache\wheels\ed\8f\46\a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [70]:
query = 'SELECT "Song", "Performer"  FROM billboard_chart WHERE "WeekID"= "1997-10-11"'

In [47]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

pysqldf(query)

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1958-0...,1958-08-02 00:00:00.000000,1,Poor Little Fool,Ricky Nelson,Poor Little FoolRicky Nelson,1,None,1,1
1,http://www.billboard.com/charts/hot-100/1995-1...,1995-12-02 00:00:00.000000,1,One Sweet Day,Mariah Carey & Boyz II Men,One Sweet DayMariah Carey & Boyz II Men,1,None,1,1
2,http://www.billboard.com/charts/hot-100/1997-1...,1997-10-11 00:00:00.000000,1,Candle In The Wind 1997/Something About The Wa...,Elton John,Candle In The Wind 1997/Something About The Wa...,1,None,1,1
3,http://www.billboard.com/charts/hot-100/2006-0...,2006-07-01 00:00:00.000000,1,Do I Make You Proud,Taylor Hicks,Do I Make You ProudTaylor Hicks,1,None,1,1
4,http://www.billboard.com/charts/hot-100/2009-1...,2009-10-24 00:00:00.000000,1,3,Britney Spears,3Britney Spears,1,None,1,1
5,http://www.billboard.com/charts/hot-100/2011-0...,2011-02-26 00:00:00.000000,1,Born This Way,Lady Gaga,Born This WayLady Gaga,1,None,1,1
6,http://www.billboard.com/charts/hot-100/2012-0...,2012-03-03 00:00:00.000000,1,Part Of Me,Katy Perry,Part Of MeKaty Perry,1,None,1,1
7,http://www.billboard.com/charts/hot-100/2014-0...,2014-09-06 00:00:00.000000,1,Shake It Off,Taylor Swift,Shake It OffTaylor Swift,1,None,1,1
8,http://www.billboard.com/charts/hot-100/2016-0...,2016-02-20 00:00:00.000000,1,Pillowtalk,Zayn,PillowtalkZayn,1,None,1,1
9,http://www.billboard.com/charts/hot-100/2016-0...,2016-05-28 00:00:00.000000,1,Can't Stop The Feeling!,Justin Timberlake,Can't Stop The Feeling!Justin Timberlake,1,None,1,1


In [31]:
billboard_df['WeekID'].dtypes

dtype('<M8[ns]')

In [5]:
df_chart = pd.read_csv('https://query.data.world/s/wndh7zyhwek7q4n2ae4hu5hyi2gaql')
df_chart.head()

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
0,http://www.billboard.com/charts/hot-100/1958-0...,8/2/1958,1,Poor Little Fool,Ricky Nelson,Poor Little FoolRicky Nelson,1,NaN,1,1
1,http://www.billboard.com/charts/hot-100/1995-1...,12/2/1995,1,One Sweet Day,Mariah Carey & Boyz II Men,One Sweet DayMariah Carey & Boyz II Men,1,NaN,1,1
2,http://www.billboard.com/charts/hot-100/1997-1...,10/11/1997,1,Candle In The Wind 1997/Something About The Wa...,Elton John,Candle In The Wind 1997/Something About The Wa...,1,NaN,1,1
3,http://www.billboard.com/charts/hot-100/2006-0...,7/1/2006,1,Do I Make You Proud,Taylor Hicks,Do I Make You ProudTaylor Hicks,1,NaN,1,1
4,http://www.billboard.com/charts/hot-100/2009-1...,10/24/2009,1,3,Britney Spears,3Britney Spears,1,NaN,1,1


In [34]:
import json

In [35]:
billboard_sample = billboard_df.iloc[:100]
result = billboard_sample.to_json(orient='records')
parsed = json.loads(result)

parsed

[{'url': 'http://www.billboard.com/charts/hot-100/1958-08-02',
  'WeekID': -360288000000,
  'Week Position': 1,
  'Song': 'Poor Little Fool',
  'Performer': 'Ricky Nelson',
  'SongID': 'Poor Little FoolRicky Nelson',
  'Instance': 1,
  'Previous Week Position': None,
  'Peak Position': 1,
  'Weeks on Chart': 1},
 {'url': 'http://www.billboard.com/charts/hot-100/1995-12-02',
  'WeekID': 817862400000,
  'Week Position': 1,
  'Song': 'One Sweet Day',
  'Performer': 'Mariah Carey & Boyz II Men',
  'SongID': 'One Sweet DayMariah Carey & Boyz II Men',
  'Instance': 1,
  'Previous Week Position': None,
  'Peak Position': 1,
  'Weeks on Chart': 1},
 {'url': 'http://www.billboard.com/charts/hot-100/1997-10-11',
  'WeekID': 876528000000,
  'Week Position': 1,
  'Song': 'Candle In The Wind 1997/Something About The Way You Look Tonight',
  'Performer': 'Elton John',
  'SongID': 'Candle In The Wind 1997/Something About The Way You Look TonightElton John',
  'Instance': 1,
  'Previous Week Position'

In [36]:
billboard_json = json.dumps(parsed, indent=4)
billboard_json

'[\n    {\n        "url": "http://www.billboard.com/charts/hot-100/1958-08-02",\n        "WeekID": -360288000000,\n        "Week Position": 1,\n        "Song": "Poor Little Fool",\n        "Performer": "Ricky Nelson",\n        "SongID": "Poor Little FoolRicky Nelson",\n        "Instance": 1,\n        "Previous Week Position": null,\n        "Peak Position": 1,\n        "Weeks on Chart": 1\n    },\n    {\n        "url": "http://www.billboard.com/charts/hot-100/1995-12-02",\n        "WeekID": 817862400000,\n        "Week Position": 1,\n        "Song": "One Sweet Day",\n        "Performer": "Mariah Carey & Boyz II Men",\n        "SongID": "One Sweet DayMariah Carey & Boyz II Men",\n        "Instance": 1,\n        "Previous Week Position": null,\n        "Peak Position": 1,\n        "Weeks on Chart": 1\n    },\n    {\n        "url": "http://www.billboard.com/charts/hot-100/1997-10-11",\n        "WeekID": 876528000000,\n        "Week Position": 1,\n        "Song": "Candle In The Wind 1997/S

In [10]:
df_chart.to_sql('billboard_chart', engine, if_exists='replace',chunksize=10000, index=False)

In [12]:
inspector = inspect(engine)
inspector.get_columns('billboard_chart')


[{'name': 'url',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'WeekID',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Week Position',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Song',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Performer',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'SongID',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Instance',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Previous Week Position',
  'type': DOUBLE_PRECISION(precision=53),
  'nullable': True,
  'default': None,
  'autoincr

In [17]:
from sqlalchemy.sql import text

In [22]:
from sqlalchemy.ext.automap import automap_base

Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# mapped classes are now created with names by default
# matching that of the table name.
chart = Base.classes

In [25]:
engine.table_names()

['billboard_chart']